In [7]:
import pandas as pd
import numpy as np
import csv
from datetime import timedelta
from dateutil import parser
import math


In [20]:
def filter2(day,month,year):
    date = str(day)+ "-"+str(month) + "-"+ str(year)
    ds = pd.read_csv("data/cleaned/data"+date+".csv",sep=",",header=None)
    cval = []
    val = []
    bv = []
    state = "00"
    c_esp = 0
    for index, row in ds.iterrows():
        cval.append(1)
        for i in range(0,2):
            x = ds.iloc[index][i]
            cval.append(x)
        for i in range(3,len(row)):
            x = ds.iloc[index][i]
            if (x !=0):
                if(index%2!=0):
                    if (x>40 and x<=60 and state =="00"):
                        state = "01"
                        val.append(x)
                        c_esp +=1
                    if (x>40 and state =="01"):
                        c_esp +=1
                        val.append(x)
                    if(x<40 and state =="01"):
                        c_esp +=1
                        val.append(x)
                        if c_esp<30:
                            val = []
                            state = "00"
                            c_esp =0
                        else:
                            state = "00"
                            break
                else:
                    val.append(x)
        c_esp =0
        state = "00"
        if (len(val)==0):
            bv.pop(index-1)
        if (len(val)!=0):
            cval.extend(val)
            bv.append(cval)
        val = []
        cval = []
    df = pd.DataFrame(bv)
    cval = []
    val = []
    bv = []
    pg =0
    pe = 0 
    for index, row in df.iterrows():
        for i in range(0,3):
            x = df.iloc[index][i]
            cval.append(x)
        for i in range(3,len(row)):
            x = df.iloc[index][i]
            if (x !=0):
                if(math.isnan(x)):
                    # appending 11 at the suffix to help the algorithm understand
                    # its the end of the line
                    val.append(11) 
                    break
                if (index%2==0):
                    if(x>pg):
                        # if the change in values is significantly positive we change it to 1
                        if (x-pg>=70):
                            val.append(1)
                    if(x<pg):
                        if(pg-x>=70):
                            # if the change in values is significantly negative we change it to -1
                            val.append(-1)
                    if(abs(x-pg)<70):
                        # if the change in values is insignificant we change it to 0
                        val.append(2)
                    pg=x
                else:
                    val.append(2)
        pg=0
        pe=0
        cval.extend(val)
        bv.append(cval)
        val = []
        cval = []

        ds2 = pd.DataFrame(bv)
    for index, row in ds2.iterrows():
        for i in range(3,len(row)):
            if(math.isnan(ds2.iloc[index][i])):
                ds2[i][index] = 0
        print(index)
    with open('finalizeddata.csv', 'a') as f:
        ds2.to_csv(f, header=False,index = False)
        